In [30]:
import numpy as np
import xarray as xr
import seawater as sw


In [2]:
model = xr.open_dataset("/Users/james/Documents/Rutgers/data/nwa25-output/19961223.ocean_5day.test.nc")
model

<xarray.Dataset>
Dimensions:     (nv: 2, xh: 1550, xq: 1551, yh: 1690, yq: 1691, z_i: 36, z_l: 35)
Coordinates:
  * xq          (xq) float64 -98.04 -98.0 -97.96 -97.92 ... -36.14 -36.1 -36.06
  * yh          (yh) float64 5.253 5.292 5.332 5.372 ... 51.91 51.92 51.93 51.94
  * z_l         (z_l) float64 2.5 10.0 20.0 32.5 ... 5e+03 5.5e+03 6e+03 6.5e+03
  * z_i         (z_i) float64 0.0 5.0 15.0 25.0 ... 5.75e+03 6.25e+03 6.75e+03
    time        object ...
  * nv          (nv) float64 1.0 2.0
  * xh          (xh) float64 -98.02 -97.98 -97.94 -97.9 ... -36.16 -36.12 -36.08
  * yq          (yq) float64 5.233 5.273 5.312 5.352 ... 51.91 51.92 51.93 51.94
Data variables:
    u           (z_l, yh, xq) float32 ...
    v           (z_l, yq, xh) float32 ...
    temp        (z_l, yh, xh) float32 ...
    salt        (z_l, yh, xh) float32 ...
    average_T1  datetime64[ns] ...
    average_T2  datetime64[ns] ...
    average_DT  timedelta64[ns] ...
    time_bnds   (nv) timedelta64[ns] ...
Attributes:
    filename:          19960828.ocean_5day.nc
    title:             NWA25_glorys_era5
    associated_files:  areacello: 19960828.ocean_static.nc
    grid_type:         regular
    grid_tile:         N/A

In [24]:

latg = 11
long = -45

depth_model = model['z_l'].values
temp_model = model['temp'].sel(xh=long, yh=latg, method='nearest').values
salt_model = model['salt'].sel(xh=long, yh=latg, method='nearest').values

In [25]:

dens_model = sw.dens(salt_model,temp_model,depth_model)

In [26]:
dens_model

array([1023.62706503, 1023.65724295, 1023.69991353, 1023.75616824,
       1023.93134795, 1024.81321159, 1025.96276254, 1026.68328925,
       1027.25550895, 1027.67465481, 1028.01691574, 1028.38204593,
       1028.85091765, 1029.37239537, 1029.90473689, 1030.46287578,
       1031.0040802 , 1031.52132899, 1032.02800926, 1032.54132291,
       1033.06954291, 1033.59201392, 1034.0986251 , 1034.77598389,
       1035.7993436 , 1037.26586961, 1039.2671392 , 1041.51273097,
       1043.74649682, 1045.96899652, 1048.13457039,           nan,
                 nan,           nan,           nan])

## Ocean Heat Content

In [27]:
def OHC_from_profile(depth,temp,dens):
    # This function Calculates the ocean heat content from a temperature and
    # density profile (Leipper, Dale F., and Douglas Volgenau. "Hurricane heat
    # potential of the Gulf of Mexico". Journal of Physical Oceanography 2.3
    #(1972): 218-224).
    # Inputs: 1D vectors depth, temperature and density
    # Output: Ocean heat content of the water column in kJ/cm^2

    cp = 3985 #Heat capacity in J/(kg K)
    ok26 = temp >= 26
    depth = np.abs(depth)

    if len(depth[ok26]) != 0:
        if np.nanmin(depth[ok26])>10:
            OHC = np.nan
        else:
            rho0 = np.nanmean(dens[ok26])
            OHC = np.abs(cp * rho0 * np.trapz(temp[ok26]-26,depth[ok26]))
            OHC = OHC * 10**(-7) # in kJ/cm^2
    else:
        OHC = np.nan
    return OHC

In [28]:
temp_model

array([26.590988 , 26.598675 , 26.601812 , 26.60397  , 26.595825 ,
       24.740465 , 21.32725  , 18.234743 , 15.51976  , 13.112597 ,
       11.412996 , 10.263714 ,  9.330247 ,  8.404127 ,  7.500722 ,
        6.707895 ,  6.11518  ,  5.710361 ,  5.452715 ,  5.2812815,
        5.058316 ,  4.8041306,  4.544073 ,  4.1832805,  3.7143085,
        3.217479 ,  2.786594 ,  2.4406657,  2.0842702,  1.7277688,
        1.6395034,        nan,        nan,        nan,        nan],
      dtype=float32)

In [32]:
OHC = OHC_from_profile(depth_model,temp_model,dens_model)
print('The OHC from NWA25 Exp 6 at (lon,lat) = (-45,11) on Deecember 26, 1996 at 00 UTC is ',\
      OHC, ' kJ/m^3 \n')

The OHC from NWA25 Exp 6 at (lon,lat) = (-45,11) on Deecember 26, 1996 at 00 UTC is  11.931912588381122  kJ/m^3 



## Mixed Layer Depth and Mixed Layer Temperature based on Temperature Criteria

In [45]:
def MLD_temp_crit(dtemp,ref_depth,depth,temp):
    # This function calculates the mixed layer depth and Mixed layer temperature
    # based on a temperature criteria: T - T_at_ref_depth <= dtemp
    # Inputs
    # dtemp: delta temperature from the mixed layer depth definition used
    # ref_depth: Reference depth from the mixed layer depth definition used
    # depth and temp: 1D vectors depth and temperature
    # Output
    # MLD and MLT: mixed layer depth and Mixed layer temperature

    ok_ref_depth = np.where(depth >= ref_depth)[0][0]
    temp_ref_depth = temp[ok_ref_depth]
    delta_T = temp_ref_depth - temp
    ok_mld_temp = np.where(delta_T <= dtemp)[0]

    if ok_mld_temp.size == 0:
        MLD = np.nan
        MLT = np.nan
    else:
        MLD = depth[ok_mld_temp[-1]]
        MLT = np.nanmean(temp[ok_mld_temp])

    return MLD, MLT


In [48]:
dtemp = 0.2
ref_depth = 10 # meters
MLD, MLT = MLD_temp_crit(dtemp,ref_depth,depth_model,\
                                             temp_model)

print('The mixed layer temperature (based on a temperatere criteria) from NWA25 Exp 6 at (lon,lat) = (-45,11) on Deecember 26, 1996 is ',\
      MLT, ' degrees C and the mixed layer depth is ', MLD, ' meters' )


The mixed layer temperature (based on a temperatere criteria) from NWA25 Exp 6 at (lon,lat) = (-45,11) on Deecember 26, 1996 is  26.598255  degrees C and the mixed layer depth is  51.25  meters


## Mixed Layer Depth and Mixed Layer Temperature based on Density Criteria

In [49]:
def MLD_dens_crit(drho,ref_depth,depth,temp,dens):
    # This function calculates the mixed layer depth and Mixed layer temperature
    # based on a density criteria: rho_at_ref_depth - rho <= drho
    # Inputs
    # drho: delta density from the mixed layer depth definition used
    # ref_depth: Reference depth from the mixed layer depth definition used
    # depth, temp and dens: 1D vectors depth, temperature and density
    # Output
    # MLD and MLT: mixed layer depth and Mixed layer temperature

    ok_ref_depth = np.where(depth >= ref_depth)[0][0]
    rho_ref_depth = dens[ok_ref_depth]
    delta_rho = -(rho_ref_depth - dens)
    ok_mld_rho = np.where(delta_rho <= drho)[0]

    if ok_mld_rho.size == 0:
        MLD = np.nan
        MLT = np.nan
    else:
        MLD = depth[ok_mld_rho[-1]]
        MLT = np.nanmean(temp[ok_mld_rho])

    return MLD, MLT

In [52]:
drho = 0.125
ref_depth = 10 # meters
MLD, MLT = MLD_dens_crit(drho,ref_depth,depth_model,\
                                             temp_model,dens_model)


print('The mixed layer temperature (based on density criteria) from NWA25 Exp 6 at (lon,lat) = (-45,11) on Deecember 26, 1996 is ',\
      MLT, ' degrees C and the mixed layer depth is ', MLD, ' meters' )

The mixed layer temperature (based on density criteria) from NWA25 Exp 6 at (lon,lat) = (-45,11) on Deecember 26, 1996 is  26.598862  degrees C and the mixed layer depth is  32.5  meters


## Average Temperature in first 100m

In [53]:
def T100(depth,temp):
    # This function calculates the depth average temperature in the top 100
    # meters
    # Inputs
    # depth, temp: 1D vectors depth and temperature
    # Output
    # T100: depth average temperature in the top 100 meters

    okd = np.abs(depth) <= 100
    if len(np.where(np.isnan(temp[okd]))[0])>10:
        T100 = np.nan
    else:
        T100 = np.nanmean(temp[okd])
    return T100

In [54]:
T100(depth_model,temp_model)

25.579855

## Potential Energy Anomaly

In [55]:
def Potential_energy_anomaly100(depth,dens):
    # This function calculates the potential energy anomaly
    # (Simpson J, Brown J, Matthews J, Allen G (1990) Tidal straining, density
    # currents and stirring in the control of estuarine stratification.
    # Estuaries 13(2):125–132), in the top 100 meters
    # Inputs
    # depth, dens: 1D vectors depth and density
    # Output
    # PEA: potential energy anomaly in J/m^3

    g = 9.8 #m/s
    dindex = np.fliplr(np.where(np.asarray(np.abs(depth)) <= 100))[0]
    if len(dindex) == 0:
        PEA = np.nan
    else:
        zz = np.asarray(np.abs(depth[dindex]))
        denss = np.asarray(dens[dindex])
        ok = np.isfinite(denss)
        z = zz[ok]
        densi = denss[ok]
        if len(z)==0 or len(densi)==0 or np.min(zz) > 10 or np.max(zz) < 30:
            PEA = np.nan
        else:
            if z[-1] - z[0] > 0:
                # So PEA is < 0
                # sign = -1
                # Adding 0 to sigma integral is normalized
                z = np.append(0,z)
            else:
                # So PEA is < 0
                # sign = 1
                # Adding 0 to sigma integral is normalized
                z = np.flipud(z)
                z = np.append(0,z)
                densit = np.flipud(densi)

            # adding density at depth = 0
            densitt = np.interp(z,z[1:],densit)
            density = np.flipud(densitt)

            # defining sigma
            max_depth = np.nanmax(zz[ok])
            sigma = -1*z/max_depth
            sigma = np.flipud(sigma)

            rhomean = np.trapz(density,sigma,axis=0)
            drho = rhomean - density
            torque = drho * sigma
            PEA = g * max_depth * np.trapz(torque,sigma,axis=0)

    return PEA

In [56]:
PEA = Potential_energy_anomaly100(depth_model,dens_model)

In [57]:
PEA

207.78461980550202